In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt

# 
from scipy import stats
from scipy.spatial.distance import cdist

# Access data using Google BigQuery.
from google.colab import auth, drive
from google.cloud import bigquery

import warnings
warnings.filterwarnings('ignore')

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [2]:
# authenticate
auth.authenticate_user()

# mount google drive
drive.mount('/content/drive')
DATA_DIR = 'drive/MyDrive/COMP90089/group_project/'

Mounted at /content/drive


In [3]:
# Set up environment variables
project_id = 'comp90089'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

In [6]:
##Module ICU stay with VITAL sign, gender, age
df_Glucose = run_query("""
SELECT 
  LAB.subject_id, LAB.stay_id, LAB.glucose_max, 
  DEMO.anchor_age, DEMO.dod, DEMO.gender, 
  VITAL.dbp_mean, VITAL.sbp_mean, VITAL.glucose_mean, VITAL.heart_rate_mean, 
  VITAL.spo2_mean, VITAL.resp_rate_mean, VITAL.temperature_mean, 
  IND.apsiii, IND.apsiii_prob, IND.glucose_score, ICUSTAY.los
FROM `physionet-data.mimiciv_derived.first_day_lab` AS LAB
INNER JOIN `physionet-data.mimiciv_hosp.patients` AS DEMO
ON LAB.subject_id = DEMO.subject_id
INNER JOIN `physionet-data.mimiciv_derived.first_day_vitalsign` AS VITAL
ON LAB.stay_id = VITAL.stay_id
INNER JOIN `physionet-data.mimiciv_derived.apsiii` AS IND
ON LAB.stay_id = IND.stay_id
INNER JOIN `physionet-data.mimiciv_icu.icustays` ICUSTAY
ON LAB.stay_id = ICUSTAY.stay_id
WHERE LAB.glucose_max >= 200
""")

# Looking at the summary statistics on numerical attributes
df_Glucose[['glucose_max', 'anchor_age', 'dbp_mean', 'sbp_mean',
           'glucose_mean', 'heart_rate_mean', 'spo2_mean',
           'resp_rate_mean', 'temperature_mean', 'apsiii',
           'glucose_score', 'los']].describe()

,glucose_max,anchor_age,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,los
count,14849.000000,14849.000000,14754.000000,14754.000000,14705.000000,14826.000000,14791.000000,14818.000000,14402.000000,14849.000000,14849.000000,14849.000000
mean,315.499697,63.351337,63.537791,119.606764,269.150999,87.870111,96.691182,19.967010,36.791170,56.653512,3.563607,4.068648
std,173.953706,16.061045,11.643143,18.017578,2885.334677,16.402283,3.122335,4.093006,0.658481,25.267327,0.934916,5.426979
min,200.000000,18.000000,17.000000,45.500000,42.000000,28.500000,26.600000,9.000000,26.670000,5.000000,3.000000,0.001551
25%,223.000000,54.000000,55.431609,106.729823,167.000000,76.175078,95.583333,17.025507,36.572292,39.000000,3.000000,1.197037
50%,260.000000,65.000000,62.666667,117.200000,197.500000,86.924501,97.150000,19.346154,36.805000,51.000000,3.000000,2.186840
75%,334.000000,75.000000,70.440000,130.965209,233.647059,98.666667,98.517880,22.351389,37.078643,69.000000,5.000000,4.541898
max,5840.000000,91.000000,129.571429,217.750000,166855.166667,166.307692,100.000000,40.880000,40.104118,184.000000,9.000000,86.312488


In [7]:
df_bmi = run_query("""
SELECT subject_id, 
  AVG(CAST(result_value AS FLOAT64)) AS avg_bmi_value
FROM `physionet-data.mimiciv_hosp.omr` 
WHERE result_name="BMI (kg/m2)"
GROUP BY subject_id, result_name
""")

# Summary Statistics on Average BMI values
df_bmi[['avg_bmi_value']].describe()

,avg_bmi_value
count,144861.000000
mean,31.508083
std,302.442950
min,0.000000
25%,23.700000
50%,27.226667
75%,31.720000
max,107840.200000


In [8]:
df_sofa = run_query("""
SELECT 
  stay_id,
  AVG(SOFA) as avg_sofa
FROM `physionet-data.mimiciv_derived.first_day_sofa` 
GROUP BY stay_id
""")

# SOFA score summary
df_sofa[['avg_sofa']].describe()

,avg_sofa
count,76943.000000
mean,4.740964
std,3.603146
min,0.000000
25%,2.000000
50%,4.000000
75%,7.000000
max,23.000000


In [9]:
df_adtType = run_query("""
SELECT ICUSTAY.stay_id, ADT.admission_type 
FROM `physionet-data.mimiciv_hosp.admissions` AS ADT
INNER JOIN `physionet-data.mimiciv_icu.icustays` AS ICUSTAY
ON ADT.hadm_id = ICUSTAY.hadm_id
""")

# Type of ICU stay summary 
df_adtType[['stay_id']].describe()

,stay_id
count,7.694300e+04
mean,3.499427e+07
std,2.888399e+06
min,3.000015e+07
25%,3.249231e+07
50%,3.499726e+07
75%,3.749059e+07
max,3.999981e+07


In [10]:
data_List = pd.DataFrame()

# Left join all remaining dataframes
data_List = pd.merge(df_Glucose, df_bmi, on=["subject_id"], how ="left")
data_List = pd.merge(data_List, df_sofa, on=["stay_id"], how="left")
data_List = pd.merge(data_List, df_adtType, on=["stay_id"], how = "left")

# data_List.head()
# Summary statistics on the final join 
# We exclude subject_id and stay_id to protect patient privacy
select_col = set(data_List .columns) - {'subject_id', 'stay_id'} 
data_List[list(select_col)].describe()

,anchor_age,apsiii,spo2_mean,los,apsiii_prob,glucose_max,dbp_mean,glucose_mean,avg_sofa,resp_rate_mean,sbp_mean,temperature_mean,glucose_score,heart_rate_mean,avg_bmi_value
count,14849.000000,14849.000000,14791.000000,14849.000000,14849.000000,14849.000000,14754.000000,14705.000000,14849.000000,14818.000000,14754.000000,14402.000000,14849.000000,14826.000000,9427.000000
mean,63.351337,56.653512,96.691182,4.068648,0.194127,315.499697,63.537791,269.150999,5.955889,19.967010,119.606764,36.791170,3.563607,87.870111,30.558477
std,16.061045,25.267327,3.122335,5.426979,0.194373,173.953706,11.643143,2885.334677,4.181948,4.093006,18.017578,0.658481,0.934916,16.402283,44.926034
min,18.000000,5.000000,26.600000,0.001551,0.014778,200.000000,17.000000,42.000000,0.000000,9.000000,45.500000,26.670000,3.000000,28.500000,3.400000
25%,54.000000,39.000000,95.583333,1.197037,0.069599,223.000000,55.431609,167.000000,3.000000,17.025507,106.729823,36.572292,3.000000,76.175078,24.325000
50%,65.000000,51.000000,97.150000,2.186840,0.116527,260.000000,62.666667,197.500000,5.000000,19.346154,117.200000,36.805000,3.000000,86.924501,27.931818
75%,75.000000,69.000000,98.517880,4.541898,0.235942,334.000000,70.440000,233.647059,8.000000,22.351389,130.965209,37.078643,5.000000,98.666667,33.114286
max,91.000000,184.000000,100.000000,86.312488,0.986072,5840.000000,129.571429,166855.166667,23.000000,40.880000,217.750000,40.104118,9.000000,166.307692,2332.250000


In [11]:
data_List.columns

Index(['subject_id', 'stay_id', 'glucose_max', 'anchor_age', 'dod', 'gender',
       'dbp_mean', 'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean',
       'resp_rate_mean', 'temperature_mean', 'apsiii', 'apsiii_prob',
       'glucose_score', 'los', 'avg_bmi_value', 'avg_sofa', 'admission_type'],
      dtype='object')

In [12]:
# Fill null for BMI
data_List['avg_bmi_value'] = data_List['avg_bmi_value'].fillna(
                                      data_List['avg_bmi_value'].mean())

# One-hot Encoding: Female = 1, Male = 0
data_List['gender'] = pd.get_dummies(data_List['gender']) ['F'] 

# if dod is NAN, replace with 0, if dod has a value, replace with 1
data_List['dod'] = data_List['dod'].notnull().astype("int") 

In [13]:
## Copy data for q1 analysis
df = data_List.copy()


## For Non-numberical columns (gender, dod): count, unique, top, freq
df.describe(include='all')

## Define functions to Remove generic outliers ##
def remove_outliers(df, col_lst):
  for col in col_lst:
    # we remove items that are outside of 99% of quantile 
    lower_q, higher_q = df[col].quantile(0.005), df[col].quantile(0.995)
    filtered_df = df[(df[col] > lower_q) & (df[col] < higher_q)]

  return filtered_df    

df_filtered = remove_outliers(df, ['glucose_max', 'anchor_age',
       'dbp_mean', 'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean',
       'resp_rate_mean', 'temperature_mean', 'los',
       'avg_bmi_value', 'avg_sofa'] )

df_final = df_filtered.dropna() # Drop Nulls
df_final[list(select_col)].describe(include='all')

# df_final.to_csv(index=False)


,anchor_age,apsiii,spo2_mean,gender,los,dod,admission_type,apsiii_prob,glucose_max,dbp_mean,glucose_mean,avg_sofa,resp_rate_mean,sbp_mean,temperature_mean,glucose_score,heart_rate_mean,avg_bmi_value
count,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000,13776.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,EW EMER.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,8373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,63.627178,56.609974,96.762735,0.459422,4.056913,0.502541,NaN,0.191327,316.198098,63.253872,269.606441,5.916667,19.920836,119.620113,36.795490,3.563226,87.835646,30.554323
std,15.970625,24.275595,2.767211,0.498369,5.283607,0.500012,NaN,0.187802,175.906612,11.359495,2959.382725,3.947002,4.043537,17.695002,0.649333,0.934397,16.280373,37.052563
min,18.000000,12.000000,51.250000,0.000000,0.016979,0.000000,NaN,0.020455,200.000000,17.000000,42.000000,1.000000,9.916667,50.863636,30.666667,3.000000,28.500000,3.400000
25%,54.000000,39.000000,95.615385,0.000000,1.261542,0.000000,NaN,0.069599,222.000000,55.365278,166.862013,3.000000,17.000000,106.831897,36.573214,3.000000,76.200000,26.214000
50%,65.000000,51.000000,97.153846,0.000000,2.218970,1.000000,NaN,0.116527,260.000000,62.385856,197.250000,5.000000,19.333333,117.081667,36.805000,3.000000,86.916667,30.558477
75%,75.000000,68.000000,98.513187,1.000000,4.551950,1.000000,NaN,0.227528,334.000000,69.885684,233.000000,8.000000,22.272727,130.755682,37.083333,5.000000,98.616711,30.558477


In [14]:
df_final = df_final[df_final["los"]<=40] ##remove los outliers
df_final.shape

(13743, 20)

In [15]:
df_final = df_final.drop(columns=["subject_id", "stay_id"]) # safe harbor principles

In [16]:
# Exporting csv files to local repo 
df_final.to_csv(DATA_DIR+'hyperglycemic_patients.csv', index=False)